In [1]:
import os
import json
import pandas as pd


In [4]:
NODE_FILE = "/playpen/jesse/drug_repurpose/PrimeKG/nodes.csv"
df_node = pd.read_csv(NODE_FILE)

unique_node_types = df_node['node_type'].nunique()
print(f"Number of unique node types: {unique_node_types}")

print("Unique node types:")
print(df_node['node_type'].unique())

print("\nCount of each node type:")
print(df_node['node_type'].value_counts())

Number of unique node types: 10
Unique node types:
['gene/protein' 'drug' 'effect/phenotype' 'disease' 'biological_process'
 'molecular_function' 'cellular_component' 'exposure' 'pathway' 'anatomy']

Count of each node type:
node_type
biological_process    28642
gene/protein          27671
disease               17080
effect/phenotype      15311
anatomy               14035
molecular_function    11169
drug                   7957
cellular_component     4176
pathway                2516
exposure                818
Name: count, dtype: int64


In [6]:
source_counts = df_node.groupby(['node_type', 'node_source']).size().reset_index(name='count')
print(source_counts)

             node_type    node_source  count
0              anatomy         UBERON  14035
1   biological_process             GO  28642
2   cellular_component             GO   4176
3              disease          MONDO  15813
4              disease  MONDO_grouped   1267
5                 drug       DrugBank   7957
6     effect/phenotype            HPO  15311
7             exposure            CTD    818
8         gene/protein           NCBI  27671
9   molecular_function             GO  11169
10             pathway       REACTOME   2516


In [10]:
EDGE_FILE = "/playpen/jesse/drug_repurpose/PrimeKG/edges.csv"  
df_edges = pd.read_csv(EDGE_FILE)

unique_relations = df_edges['relation'].nunique()
print(f"Number of unique relation types: {unique_relations}")

print("\nUnique relation types:")
print(df_edges['relation'].unique())

print("\nRelation counts:")
print(df_edges['relation'].value_counts())

Number of unique relation types: 30

Unique relation types:
['protein_protein' 'drug_protein' 'contraindication' 'indication'
 'off-label use' 'drug_drug' 'phenotype_protein' 'phenotype_phenotype'
 'disease_phenotype_negative' 'disease_phenotype_positive'
 'disease_protein' 'disease_disease' 'drug_effect' 'bioprocess_bioprocess'
 'molfunc_molfunc' 'cellcomp_cellcomp' 'molfunc_protein'
 'cellcomp_protein' 'bioprocess_protein' 'exposure_protein'
 'exposure_disease' 'exposure_exposure' 'exposure_bioprocess'
 'exposure_molfunc' 'exposure_cellcomp' 'pathway_pathway'
 'pathway_protein' 'anatomy_anatomy' 'anatomy_protein_present'
 'anatomy_protein_absent']

Relation counts:
relation
anatomy_protein_present       3036406
drug_drug                     2672628
protein_protein                642150
disease_phenotype_positive     300634
bioprocess_protein             289610
cellcomp_protein               166804
disease_protein                160822
molfunc_protein                139060
drug_effect

In [28]:
# Inference path relations:
needed_relations = [
    "disease_protein",
    "drug_effect",
    "drug_protein",
    "disease_phenotype_positive"
]

# Validation relation:
validation_relation = "indication"

In [37]:
path_edges = df_edges[df_edges['relation'].isin(needed_relations)]

disease_gene = path_edges[path_edges['relation'] == 'disease_protein'][['x_index', 'y_index']]
drug_gene = path_edges[path_edges['relation'] == 'drug_protein'][['x_index', 'y_index']]
disease_pheno = path_edges[path_edges['relation'] == 'disease_phenotype_positive'][['x_index', 'y_index']]
drug_pheno = path_edges[path_edges['relation'] == 'drug_effect'][['x_index', 'y_index']]

disease_gene.columns = ['gene_index', 'disease_index']
drug_gene.columns = ['drug_index', 'gene_index']
disease_pheno.columns = ['disease_index', 'phenotype_index']
drug_pheno.columns = ['drug_index', 'phenotype_index']
print(disease_gene)

         gene_index  disease_index
3235582        7097          28313
3235583        2174          28313
3235584        8038          28313
3235585        5925          28313
3235586         238          28313
...             ...            ...
6030122       28149           4152
6030123       28181             59
6030124       31190           5826
6030125       33606          10422
6030126       30032          33802

[160822 rows x 2 columns]


In [53]:
# ---- Inference Path 1: Disease → Protein → Drug ----
inferred_disease_drug_1 = pd.merge(disease_gene, drug_gene, on='gene_index')

# ---- Inference Path 2: Disease → Phenotype → Drug ----
inferred_disease_drug_2 = pd.merge(disease_pheno, drug_pheno, on='phenotype_index')

print(inferred_disease_drug_1)
print(inferred_disease_drug_2)

        gene_index  disease_index  drug_index
0             7097          28313       14679
1             7097          28313       14012
2             7097          28313       14680
3             2174          28313       15852
4             2174          28313       14280
...            ...            ...         ...
563726        4152          28149       14499
563727        4152          28149       14050
563728        4152          28149       15752
563729        5826          31190       14490
563730        5826          31190       14491

[563731 rows x 3 columns]
         disease_index  phenotype_index  drug_index
0                27158            22784       14881
1                27158            22784       14537
2                27158            22784       14140
3                27158            22272       14544
4                27158            22272       14275
...                ...              ...         ...
6418453          32207            22574       15432
64184

In [41]:
# Load known indications
indication_edges = df_edges[df_edges['relation'] == validation_relation][['x_index', 'y_index']]
indication_edges.columns = ['drug_index', 'disease_index']
print(indication_edges)

         drug_index  disease_index
346730        16687          33577
346731        16687          36035
346764        20297          33577
346765        20297          36035
346768        16693          33577
...             ...            ...
5776153       84333          14471
5776154       27527          16634
5776155       38622          16634
5776156       28673          16634
5776158       39497          17237

[18776 rows x 2 columns]


In [48]:
drug_disease_proposed = inferred_pairs[inferred_pairs['drug_index'] == 16687]['disease_index']
drug_disease_proposed_list = drug_disease_proposed.tolist()
print(len(drug_disease_proposed_list), drug_disease_proposed_list)

drug_disease_actual = indication_edges[indication_edges['drug_index'] == 16687]['disease_index']
drug_disease_actual_list = drug_disease_actual.tolist()
print(len(drug_disease_actual_list), drug_disease_actual_list)

3509 [28313, 28780, 83960, 38575, 83911, 28473, 27933, 37703, 83760, 31116, 36812, 28445, 29646, 36634, 83869, 32621, 36999, 83870, 37115, 30035, 35966, 30826, 35963, 36187, 38044, 33603, 35953, 84310, 31912, 37717, 36040, 83923, 83924, 33653, 94633, 35979, 39850, 39857, 94634, 37324, 33598, 36432, 37659, 38242, 28249, 37718, 36312, 83841, 83840, 38957, 37180, 83823, 83824, 35958, 29945, 36000, 31380, 83899, 37273, 36242, 84131, 83910, 36286, 38388, 28827, 36694, 36968, 31459, 32831, 35686, 38941, 84045, 33618, 83795, 37549, 37787, 28772, 27292, 39733, 94844, 33390, 94845, 33605, 27630, 27845, 27847, 27337, 27851, 27855, 27858, 27860, 27862, 27864, 27865, 27530, 27721, 27867, 27869, 27813, 27873, 27875, 27215, 27877, 27878, 27164, 27618, 27884, 27886, 27887, 27890, 27894, 27895, 27896, 27900, 27903, 27450, 27904, 27907, 27911, 27913, 27914, 27853, 27461, 27917, 27167, 27920, 27922, 27926, 27169, 27402, 27929, 27931, 27207, 27368, 27315, 27938, 27944, 27172, 27351, 27173, 27954, 27955, 

In [50]:
for ddp in drug_disease_proposed_list:
    if ddp in drug_disease_actual_list:
        print("Yes")
        print(ddp)

Yes
33577


         x_index  y_index
346730     16687    33577
346731     16687    36035
346764     20297    33577
346765     20297    36035
346768     16693    33577
...          ...      ...
5776153    84333    14471
5776154    27527    16634
5776155    38622    16634
5776156    28673    16634
5776158    39497    17237

[18776 rows x 2 columns]
Total inferred pairs: 406600
Validated hits (via indication): 1931

Sample validated hits (undirected):
           entity_1       entity_2
121     Ziprasidone  schizophrenia
125      Olanzapine  schizophrenia
130        Loxapine  schizophrenia
131       Promazine  schizophrenia
136  Chlorpromazine  schizophrenia
